In [14]:
import catboost
import pandas as pd
import numpy as np
from typing import List
import pandas as pd
from scipy import stats
import datetime
import rpy2
from functools import reduce
from catboost import CatBoostRegressor
import time
import seaborn as sns
import ast
import numpy as np
from dateutil import parser
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import catboost

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

In [15]:
# Path fo import files
#input data
import_data = "import/CDNOW_master.txt"
#generated data
transformed_analyzed_data = 'export/cdnow/cdnow_initial_time_series_2000.csv'
monetary_time_series = "export/cdnow/cdnow_monetary_2000.csv"
frequency_time_series = "export/cdnow/cdnow_frequency_2000.csv"
recency_time_series = "export/cdnow/cdnow_recency_2000.csv"
cluster_centroids_time_series = "export/cdnow/cdnow_cluster_centroids_2000.csv"
#prediction
prediction_path = "export/cdnow/cdnow_prediction.csv"

## 0. Preprocessing cdnow

In [16]:
df = pd.read_csv(import_data,header=None,names  =['id','Date','disks','price_purchase'],delim_whitespace=True,)

In [17]:
df['Date'] = df["Date"].apply(lambda x:datetime.datetime.strptime(str(x),"%Y%m%d"))
df = df.drop(columns = ['disks'])
df = df.groupby(by=['id','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'id',values = 'price_purchase')
df.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
16582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.1 Taking random 2000 users

In [18]:
df_test = df.sample(2000)
df_test.to_csv(transformed_analyzed_data)

## 1 RFM-analysis 

In [19]:
df_recency =  pd.DataFrame(columns = df_test.columns)
df_monetary =  pd.DataFrame(columns = df_test.columns)
df_frequency = pd.DataFrame(columns = df_test.columns)

In [ ]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df_test.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry)):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
    #MONETARY  Moving average monetary value
    current_servers = 0
    total_days_of_purchases = 0 
    first_purchase = -1#flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_days_of_purchases/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

## 2. Clustering

In [ ]:
ksh_monetary = KShape(n_clusters=4)
df_monetary['label'] = ksh_monetary.fit(df_monetary.values)
ksh_frequency = KShape(n_clusters=4)
df_frequency['label'] = ksh_frequency.fit(df_frequency.values)
ksh_recency = KShape(n_clusters=4)
df_recency['label'] = ksh_recency.fit(df_recency.values)

In [ ]:
df_monetary['label'] =ksh_monetary.labels_
df_frequency['label'] =ksh_frequency.labels_
df_recency['label'] =ksh_recency.labels_

In [ ]:
df_centroids = pd.DataFrame(columns = df_recency.drop(columns = 'label').columns)
for i in range(ksh_recency.cluster_centers_.shape[0]):
    df_centroids.loc['Centroid_for_recency_'+str(i)]= ksh_recency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_frequency_'+str(i)]= ksh_frequency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_monetary_'+str(i)]= ksh_monetary.cluster_centers_[i].ravel()

In [ ]:
df_monetary.to_csv(monetary_time_series)
df_frequency.to_csv(frequency_time_series)
df_recency.to_csv(recency_time_series)
df_centroids.to_csv(cluster_centroids_time_series)

## 3. Preprocessing for prediction

In [2]:
df = pd.read_csv(transformed_analyzed_data)
df.head()

,id,1997-01-01,1997-01-02,1997-01-03,1997-01-04,1997-01-05,1997-01-06,1997-01-07,1997-01-08,1997-01-09,...,1998-06-21,1998-06-22,1998-06-23,1998-06-24,1998-06-25,1998-06-26,1998-06-27,1998-06-28,1998-06-29,1998-06-30
0,6383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.melt(df,id_vars = ['id'],var_name ='timestamp' ,value_name='value')
df.head()

,id,timestamp,value
0,6383,1997-01-01,NaN
1,6146,1997-01-01,NaN
2,5044,1997-01-01,NaN
3,10377,1997-01-01,NaN
4,6496,1997-01-01,NaN


In [11]:
df_freq = pd.read_csv(frequency_time_series).loc[:,['Unnamed: 0','label']]
df_monet = pd.read_csv(monetary_time_series).loc[:,['Unnamed: 0','label']]
df_recency = pd.read_csv(recency_time_series).loc[:,['Unnamed: 0','label']]

In [12]:
df_monet.head()

,Unnamed: 0,label
0,6383,3
1,6146,3
2,5044,3
3,10377,3
4,6496,3


In [13]:
df_combined = df.merge(df_recency,left_on = 'id',right_on='Unnamed: 0') \
    .merge(df_freq,left_on = 'id',right_on='Unnamed: 0') \
    .merge(df_monet,left_on = 'id',right_on='Unnamed: 0') \
    .drop(columns =['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])
df_combined.columns = ['id','timestamp','value','recency','frequency','monetary']
df_combined.sample(5)

KeyError: 'id'

In [ ]:
#For CDNow
df_combined['timestamp'] = df_combined['timestamp'].apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d"))
df_combined.sample(2)

In [ ]:
df_combined = df_combined.fillna(0)

In [ ]:
# number of months
test_period = 4
current_date = df_combined.timestamp.max()
df_test = df_combined[df_combined.timestamp >= current_date - pd.DateOffset(months = test_period )]
df_train = df_combined[df_combined.timestamp < current_date - pd.DateOffset(months = test_period )]

In [ ]:
df_test.shape[0]/df_train.shape[0]

In [11]:
y_train = df_train['value']
y_test = df_test['value']

X_train = df_train.drop('value', axis = 1)
X_test = df_test.drop('value', axis = 1)

## 4. Prediction

In [12]:
start_time = time.time()

cv = KFold(n_splits = 3, shuffle = True, random_state = 42)  

model = CatBoostRegressor(verbose = False, 
                           random_state=42)

param_grid = {
              'n_estimators': [200, 300],
              'max_depth': [3, 4], 
              'learning_rate': [0.03, 0.05, 0.07] 
             }

gs_cb = GridSearchCV(model, 
                  param_grid,
                  cv = cv,
                  scoring = 'neg_mean_squared_error')

gs_cb.fit(X_train, y_train)

#train_RMSE = np.sqrt(-gs_cb.cv_results_['mean_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
#train_std = np.sqrt(gs_cb.cv_results_['std_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

cv_RMSE = np.sqrt(-gs_cb.cv_results_['mean_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
cv_std = np.sqrt(gs_cb.cv_results_['std_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

test_RMSE = np.sqrt(mean_squared_error(gs_cb.predict(X_test), y_test))

#print('train RMSE: {} +/- {}'.format(np.round(train_RMSE, 2), np.round(train_std, 2)))
print('CV RMSE: {} +/- {}'.format(np.round(cv_RMSE, 2), np.round(cv_std, 2)))
print('test RMSE: {}'.format(np.round(test_RMSE, 2)))
print(gs_cb.best_params_)
print('time :{} min'.format(np.round((time.time() - start_time)/60, 2))) 

filename = 'CatBoost_cdnow.sav'
pickle.dump(gs_cb, open(filename, 'wb'))

CV RMSE: 3.63 +/- 1.43
test RMSE: 3.0
{'learning_rate': 0.07, 'max_depth': 4, 'n_estimators': 300}
time :14.57 min


In [13]:
result = X_test
result['predicted_value']=gs_cb.predict(X_test)
result['true_value']=y_test


result.to_csv(prediction_path)